In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/glove-twitter/glove.twitter.27B.100d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.200d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.50d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.25d.txt


In [2]:
# Load the competetion datasets

class KaggleReader(object):
    def __init__(self):
        self.df_objects = {}        
    def read_kaggle_df(self, df_name, df_path):
        self.df_objects[df_name] = pd.read_csv(df_path)
        print()
        print(df_name + ' loaded')
        print("Shape => " + str(self.df_objects[df_name].shape))
    
df_objs = KaggleReader()
df_objs.read_kaggle_df('train', "../input/tweet-sentiment-extraction/train.csv")
df_objs.read_kaggle_df('test', "../input/tweet-sentiment-extraction/test.csv")
df_objs.read_kaggle_df('submission', "../input/tweet-sentiment-extraction/sample_submission.csv")


train loaded
Shape => (27481, 4)

test loaded
Shape => (3534, 3)

submission loaded
Shape => (3534, 2)


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from numpy import hstack, vstack
# Tokenize text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from nltk.tokenize import WordPunctTokenizer

# Model
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Embedding, Dropout, Input, Bidirectional, TimeDistributed, BatchNormalization
from keras.initializers import Constant
from keras.layers.merge import concatenate
import keras

from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [5]:
class Handlers(object):
    def __init__(self):
        self.run_tests()
        pass
    
    def draw_pr_curve_plt(self, Y_valid, y_pred, x_range=1.0):
    #     (precision, recall, x_range=1.0):

            precision, recall, thresholds_pr = precision_recall_curve(Y_valid, y_pred)

            # import dependencies
            import matplotlib.pyplot as plt

            plt.step(recall, precision, color='b', alpha=0.2,
                     where='post')
            plt.fill_between(recall, precision, alpha=0.2, color='b')
            plt.xlabel('Recall')
            plt.ylabel('Precision')
            plt.ylim([0.0, 1.05])
            plt.xlim([0.0, x_range])
            plt.show()
     
    def draw_roc_curve_plt(self, Y_valid, y_pred):
#     (fpr, tpr, auc):
        # import dependencies
        import matplotlib.pyplot as plt
        fpr, tpr, thresholds_roc = roc_curve(Y_valid, y_pred)
        auc_roc = auc(fpr, tpr)
        
        plt.figure(1)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr, tpr, label='(area = {:.3f})'.format(auc_roc))
        # plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
        plt.title('ROC curve')
        plt.legend(loc='best')
        plt.show()

    # Optimize using KMP
    def return_start_end_indices(self, big, small):
        if len(small) <= 0 or len(big) <= 0:
            return (-1,-1)
        i = 0
        j = 0
        started = False
        while i < len(big):
            if big[i] == small[j]:
                if started == False:
                    started = True
                    i_start = i

                if j == len(small) - 1:
                    return (i-len(small)+1, i)
                j += 1

            else:
                if started == True:
                    started = False
                    i = i_start + 1
                j = 0
            i += 1
        return (-1,-1)
    
    def run_tests(self):
        
        assert self.return_start_end_indices(['i', '`', 'd', 'have', 'responded', ',', 'if', 'i', 'were', 'going'], ['have', 'responded', ',']) == (3,5)
        assert self.return_start_end_indices('abc', '') == (-1,-1)
        assert self.return_start_end_indices('abc', 'a') == (0,0)
        assert self.return_start_end_indices('abc', 'b') == (1,1)
        assert self.return_start_end_indices('abc', 'c') == (2,2)
        assert self.return_start_end_indices('abc', 'ab') == (0,1)
        assert self.return_start_end_indices('abc', 'bc') == (1,2)
        assert self.return_start_end_indices('abc', 'ac') == (-1,-1)
        assert self.return_start_end_indices('abc', 'abc') == (0,2)
        assert self.return_start_end_indices('abcabcabc', 'abc') == (0,2)
        assert self.return_start_end_indices('ababcc', 'abc') == (2,4)
    
    def jaccard(self, str1, str2): 
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    
    def load_gloVe_embeddings(self):
        # Boilerplate taken from here - https://www.kaggle.com/stacykurnikova/using-glove-embedding
        embeddings_index = {}
        f = open('/kaggle/input/glove-twitter/glove.twitter.27B.25d.txt')
        for line in f:
            values = line.split(' ')
            word = values[0] ## The first entry is the word
            coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
            embeddings_index[word] = coefs
        f.close()
        print('GloVe data loaded')
        return embeddings_index
    
    def load_embeddings_matrix(self, embeddings_index, index_tokenizer):
        # https://www.kaggle.com/stacykurnikova/using-glove-embedding
        # Create an embedding matrix with embedding vectors for the tokens recognized in the vocab of tweets
        
        EMBEDDING_DIM = embeddings_index.get('a').shape[0]
        # num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
        num_words = len(index_tokenizer.word_index) + 1

        # To Do: constrain the vocab size
        embedding_matrix = np.random.uniform(-1,+1,(num_words, EMBEDDING_DIM))
        count_in_embedding_vocab = 0
        for word, i in index_tokenizer.word_index.items():
            embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
                count_in_embedding_vocab += 1
#         TO DO:
#             else:
#                 PASS EMBEDDINGS OF SYNONYMS (!!! == !)
        return embedding_matrix
        # print(count_in_embedding_vocab)
        # print(len(embedding_matrix))
    
    def get_token_indices(self, x):
        span_generator = WordPunctTokenizer().span_tokenize(x)
        spans = [span for span in span_generator]
        return spans
    
    # TO DO: compensate for end and start on top of padding
    def get_pred_text_span(self, x):
        start_token_index = int(x['pred_start'])
        end_token_index = int(x['pred_end'])
        token_indices_list = x['tokens_indices']
        text = x['text']

        # start token ind > end token ind (could change logic to also have either 0:end)
        if start_token_index > end_token_index:
            return text

        # start token ind and end token ind within bounds
        elif start_token_index < len(token_indices_list) and end_token_index < len(token_indices_list):
            return text[token_indices_list[start_token_index][0]: token_indices_list[end_token_index][1]]


        # start token ind after bounds (could change logic to also have either 0:end or sth similar)
        elif start_token_index >= len(token_indices_list):
            return text

        # only end token ind out of bounds
        elif start_token_index < len(token_indices_list) and end_token_index >= len(token_indices_list):
            return text[token_indices_list[start_token_index][0]: len(text)-1]
        
    def get_preds_out(self, preds_indexes, test_df):
        temp_df = pd.concat([test_df, pd.DataFrame(preds_indexes, columns=['pred_start', 'pred_end'])], axis=1)
        temp_df['tokens_indices'] = temp_df['trans_text'].apply(handlers.get_token_indices)
        # print((temp_df['tokens_indices'].apply(len) == temp_df['tokens'].apply(len)).value_counts())

        # Get the predictions
        return temp_df.apply(handlers.get_pred_text_span, axis = 1)
    
    def get_indexes_from_argmax(self, preds, test_df):
        start_preds = preds[0]
        end_preds = preds[1]
        start_preds = start_preds.reshape((start_preds.shape[0], start_preds.shape[1]))
        end_preds = end_preds.reshape((end_preds.shape[0], end_preds.shape[1]))
        start_inds = start_preds.argmax(axis=1)
        end_inds = end_preds.argmax(axis=1)
        return vstack([start_inds, end_inds]).transpose()
        
        
    def batch_jaccard(self, preds, test_df):
        temp_df = pd.concat([test_df, pd.DataFrame(preds, columns=['pred_start', 'pred_end'])], axis=1)
        temp_df['tokens_indices'] = temp_df['trans_text'].apply(handlers.get_token_indices)
        # print((temp_df['tokens_indices'].apply(len) == temp_df['tokens'].apply(len)).value_counts())

        # Get the predictions
        temp_df['out_pred_span'] = temp_df.apply(handlers.get_pred_text_span, axis = 1)
        return temp_df.apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean()
    
    
    def batch_jaccard_from_argmax(self, preds, test_df):
        start_preds = preds[0]
        end_preds = preds[1]
        start_preds = start_preds.reshape((start_preds.shape[0], start_preds.shape[1]))
        end_preds = end_preds.reshape((end_preds.shape[0], end_preds.shape[1]))
        start_inds = start_preds.argmax(axis=1)
        end_inds = end_preds.argmax(axis=1)
        return self.batch_jaccard(vstack([start_inds, end_inds]).transpose(), test_df)    

handlers = Handlers()


# To add Jaccard Similarity Value on Validation set after each epoch
class Metrics(keras.callbacks.Callback):
    # https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, X_sentiment, y_val = self.validation_data[0], self.validation_data[1], self.validation_data[2]
        y_predict = np.asarray(self.model.predict([X_val, X_sentiment]))
        print('Val Jaccard Similarity: {}'.format(handlers.batch_jaccard(y_predict, test_df))) 
        return

    def get_data(self):
        return self._data
    
class MetricsCategorical(keras.callbacks.Callback):
    # https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, X_numerics, y_val = self.validation_data[0], self.validation_data[1], self.validation_data[2]
        y_predict = np.asarray(self.model.predict([X_val, X_numerics]))
        print('Val Jaccard Similarity: {}'.format(handlers.batch_jaccard_from_argmax(y_predict, test_df))) 
        return

    def get_data(self):
        return self._data    
    
metrics = Metrics()
metrics_categorical = MetricsCategorical()

In [6]:
MAX_SEQUENCE_LENGTH = 40

class DataManipulationPipeline(object):
    def __init__(self):
        # Load dependencies
        try:
            self.handlers = handlers
        except NameError:
            self.handlers = Handlers()
            
        # init vars
        self.vars = {
            'MAX_SEQUENCE_LENGTH' : MAX_SEQUENCE_LENGTH
        }

    def pre_fit(self, X):
        
        # Strip the text
        X['text'] = X['text'].str.strip()
        # Lower case the text
        X['trans_text'] = X['text'].apply(str.lower)
        return X

    def create_sequences(self, X):
        # Tokenize the word tokens to word_indexes
        sequences = self.vars['keras_index_tokenizer'].texts_to_sequences(X['tokens'])
        # Pad the sequences to be fed to NN [Note that this will effectively change the start, end index if padded on post]
        sequences_padded = pad_sequences(sequences, maxlen=self.vars['MAX_SEQUENCE_LENGTH'], padding='post', truncating='post')        

        return sequences_padded
    
    def create_Y(self, X):
        X['trans_selected_text'] = X['selected_text'].apply(str.lower)
        X = self.punct_tokenize(X, 'trans_selected_text', 'tokens_selected_text', self.vars['punct_tokenizer'])
        X['start_end_indices'] = X.apply(lambda x: handlers.return_start_end_indices(x['tokens'], x['tokens_selected_text']), axis=1)
        # truncate the start end indices to end of MAX LEN of sequence
        X['start_end_indices'] = X['start_end_indices'].apply(lambda x: (x[0], (MAX_SEQUENCE_LENGTH-1) if x[1]>= MAX_SEQUENCE_LENGTH else x[1]))
        # if start is ahead of end, put stamp
        X['start_end_indices'] = X['start_end_indices'].apply(lambda x: (x[0],x[1]) if x[0] <= x[1] else (-1,-1))
        
        X = X[(X['start_end_indices'] != (-1,-1))]
        X = X.reset_index(drop=True)
        X['start_ind'] = X['start_end_indices'].apply(lambda x: x[0])
        X['end_ind'] = X['start_end_indices'].apply(lambda x: x[1])
        Y = hstack(
            (
                X['start_ind'].values.reshape(X.shape[0],1),
                X['end_ind'].values.reshape(X.shape[0],1)
            )
        )
        return Y,X
    
    def create_Y_Categorical(self, Y):
        Y_categorical = np.array([[0]*MAX_SEQUENCE_LENGTH for j in range(Y.shape[0])])
        for i in range(Y.shape[0]):
            for j in range(Y[i][0], Y[i][1]+1):
                Y_categorical[i][j] = 1
        
        return keras.utils.to_categorical(Y_categorical, 2)
    
    def create_Y_Categorical_two_outs(self, Y):
        Y_categorical_start = np.array([[0]*MAX_SEQUENCE_LENGTH for j in range(Y.shape[0])])
        for i in range(Y.shape[0]): # for each example
            # print(Y[i])
            Y_categorical_start[i][Y[i][0]] = 1
        Y_categorical_start = Y_categorical_start.reshape(Y_categorical_start.shape[0], Y_categorical_start.shape[1], 1)

        Y_categorical_end = np.array([[0]*MAX_SEQUENCE_LENGTH for j in range(Y.shape[0])])
        for i in range(Y.shape[0]): # for each example
            Y_categorical_end[i][Y[i][1]] = 1
        Y_categorical_end = Y_categorical_end.reshape(Y_categorical_end.shape[0], Y_categorical_end.shape[1], 1)            

#         return (keras.utils.to_categorical(Y_categorical_start, 2), keras.utils.to_categorical(Y_categorical_end, 2))
        return (Y_categorical_start, Y_categorical_end)
        
    
    def handle_sentiment_feature_eng(self, X):
        sentiment_transform = self.vars['sentiment_one_hot'].transform(X['sentiment'].values.reshape((X['sentiment'].shape[0],1)))
        # Copy over each data row for MAX_SEQUENCE_LENGTH times to send it inside each LSTM sequence
#         sentiment_transform_repeated = np.array([([sentiment_transform[i] for x in range(MAX_SEQUENCE_LENGTH)]) for i in range(X.shape[0])])
        
        # To Do: Scale?
        return sentiment_transform
    
    def text_stats(self, X):
        X['char_len'] = X['trans_text'].apply(len)
        X['word_len'] = X['tokens'].apply(len)
        X['char_word_ratio'] = X['char_len']/X['word_len']
            
        text_stats_singular = X[['char_len', 'word_len', 'char_word_ratio']].values
        
        # Standardize
#         text_stats_singular        

#         text_stats_repeated = np.array([([text_stats_singular[i] for x in range(MAX_SEQUENCE_LENGTH)]) for i in range(X.shape[0])])
        return text_stats_singular

    def repeater(self, X):
        return np.array([([X[i] for x in range(MAX_SEQUENCE_LENGTH)]) for i in range(X.shape[0])])
    
        
    
    # Fits the parameters on train
    def fit_transform(self, X):
        
        # {x}
        # Remove nulls from train
        X = X.dropna()
        
        X = self.pre_fit(X)
        
        # Create word tokens from sentences using NTLK
        self.vars['punct_tokenizer'] = WordPunctTokenizer()
        # put sth else inseatd of X foe expanded vocab?
        X = self.punct_tokenize(X, 'trans_text', 'tokens', self.vars['punct_tokenizer'])

        # Fit Index-Tokenizer the vocab using Keras
        self.vars['keras_index_tokenizer'] = Tokenizer()
        # Use all words for extended voacb
        self.vars['keras_index_tokenizer'].fit_on_texts(
            pd.concat(
            [
                df_objs.df_objects['train']['text'].dropna().str.strip().apply(str.lower),
                # To Do: Make an option statement on test in case not available
                # Increase Vocab
                df_objs.df_objects['test']['text'].dropna().str.strip().apply(str.lower)
            ]
        ).reset_index(drop=True).apply(WordPunctTokenizer().tokenize)
        )
        # len(keras_tokenizer.word_index)
        
        # load glove embeddings
        self.vars['embeddings_index'] = self.handlers.load_gloVe_embeddings() 
        self.vars['embeddings_matrix'] = self.handlers.load_embeddings_matrix(self.vars['embeddings_index'], self.vars['keras_index_tokenizer'])

        # {y}
        # Create label column - Word Tokenize selected_text, and create label indices
        Y,X = self.create_Y(X)
        Y_categorical = self.create_Y_Categorical_two_outs(Y)
        # Run final X transform after Y since Y transform filters out some X
        sequences_padded = self.create_sequences(X)
        
        # Fit one hot encoder for sentiment (Try standard scalar downstream as well?)
        self.vars['sentiment_one_hot'] = OneHotEncoder(handle_unknown='ignore', sparse=False)
        self.vars['sentiment_one_hot'].fit(X['sentiment'].values.reshape((X['sentiment'].shape[0],1)))
        # print(enc.get_feature_names())
        X_sentiment_one_hot = self.handle_sentiment_feature_eng(X)
        
        # Create text stats
        X_text_stats = self.text_stats(X)
        self.vars['X_text_stats_standard_scaler'] = StandardScaler()
        self.vars['X_text_stats_standard_scaler'].fit(X_text_stats)
        X_text_stats_scaled = self.vars['X_text_stats_standard_scaler'].transform(X_text_stats)
        X_agg_numerics_all = hstack([X_sentiment_one_hot, X_text_stats_scaled])
        
        X_numerics_repeated = self.repeater(X_agg_numerics_all)
        
        return (sequences_padded, X_numerics_repeated, Y, Y_categorical, X)


    # Transforms using the parameters on train
    def transform(self, X):
        
        X = self.pre_fit(X)
        
        # Create word tokens
        X = self.punct_tokenize(X, 'trans_text', 'tokens', self.vars['punct_tokenizer'])
        
        Y = None
        Y_categorical = None
        if 'selected_text' in X.columns:
            Y,X = self.create_Y(X)
            Y_categorical = self.create_Y_Categorical_two_outs(Y)
        sequences_padded = self.create_sequences(X)
        X_sentiment_one_hot = self.handle_sentiment_feature_eng(X)        
        # Create text stats
        X_text_stats = self.text_stats(X)
        X_text_stats_scaled = self.vars['X_text_stats_standard_scaler'].transform(X_text_stats)
        
        X_agg_numerics_all = hstack([X_sentiment_one_hot, X_text_stats_scaled])
        X_numerics_repeated = self.repeater(X_agg_numerics_all)
    
        
        return (sequences_padded, X_numerics_repeated, Y, Y_categorical, X)
        
        
    def punct_tokenize(self, X,old_col, new_col, punct_tokenizer):
        X[new_col] = X[old_col].apply(punct_tokenizer.tokenize)
        return X

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_objs.df_objects['train'], test_size=0.15, random_state=42)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
# X_train, X_test, Y_train, Y_test, idx1, idx2 = train_test_split(df_objs.df_objects['train'], Y, np.arange(Y.shape[0]), test_size=0.15, random_state=42)


In [8]:
data_mainpulation_pipeline = DataManipulationPipeline()
(train_manip_X, train_numerics_repeated, train_manip_Y, train_manip_Y_categorical, train_df) = data_mainpulation_pipeline.fit_transform(train)
(test_manip_X, test_numerics_repeated, test_manip_Y, test_manip_Y_categorical, test_df) = data_mainpulation_pipeline.transform(test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

GloVe data loaded


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [10]:
VOCAB_SIZE = len(data_mainpulation_pipeline.vars['keras_index_tokenizer'].word_index)+1
EMBEDDING_DIM = len(data_mainpulation_pipeline.vars['embeddings_index']['a'])

In [14]:
# Model
# ----------------------------------------------------
inputLayer_words = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputLayer_agg_numerics = Input(shape=(
    MAX_SEQUENCE_LENGTH,
    train_numerics_repeated.shape[2]
))

# Embedding layer for the tap names
wordEmbeddings = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputLayer_words)
wordEmbeddings = Embedding(
    input_dim=VOCAB_SIZE, 
    output_dim=EMBEDDING_DIM, 
    input_length=MAX_SEQUENCE_LENGTH, 
    weights=[data_mainpulation_pipeline.vars['embeddings_matrix']],
    trainable=False)(inputLayer_words)
# 
merged_Input = concatenate([wordEmbeddings, inputLayer_agg_numerics])
# inputLayer_agg_indices
# # LSTM
lstm_1 = Bidirectional(LSTM(50, 
                            return_sequences = True,
                            dropout=0.2,
                            recurrent_dropout=0.2,
                           ))(merged_Input)
# b_norm_1 = BatchNormalization(5)(lstm_1)
# input_shape=(25,(EMBEDDING_DIM))
td = TimeDistributed(Dense(25))(lstm_1)


# drp1 = Dropout(0.2)(lstm_1)

# # Dense
# dense_0 = Dense(75, activation='relu')(drp1)
# drp2 = Dropout(0.2)(dense_0)
# dense_1 = Dense(30, activation='relu')(drp2)
outputLayer_start = Dense(1, activation='sigmoid')(td)
outputLayer_end = Dense(1, activation='sigmoid')(td)

model_2 = Model(inputs=[inputLayer_words, inputLayer_agg_numerics], outputs=[outputLayer_start, outputLayer_end])

model_2.compile(loss='binary_crossentropy', optimizer='adam')
# model.compile(loss='mse', optimizer='adam', metrics=[jaccard])
model_2.summary()
# ----------------------------------------------------

model_2.fit(
    [train_manip_X, train_numerics_repeated], 
    [train_manip_Y_categorical[0], train_manip_Y_categorical[1]], 
    validation_data=(
        [test_manip_X, test_numerics_repeated], 
        [test_manip_Y_categorical[0], test_manip_Y_categorical[1]]), 
    batch_size=100, 
    epochs=200,
    callbacks=[metrics_categorical]
)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 40, 25)       728175      input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 40, 6)        0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 40, 31)       0           embedding_6[0][0]                
                                                                 input_6[0][0]              

Val Jaccard Similarity: 0.6999335772331678
Epoch 25/200
21560/21560 [==============================] - 12s 578us/step - loss: 0.0755 - dense_8_loss: 0.0382 - dense_9_loss: 0.0373 - val_loss: 0.0699 - val_dense_8_loss: 0.0357 - val_dense_9_loss: 0.0351
Val Jaccard Similarity: 0.6985261175492206
Epoch 26/200
21560/21560 [==============================] - 13s 587us/step - loss: 0.0751 - dense_8_loss: 0.0380 - dense_9_loss: 0.0371 - val_loss: 0.0697 - val_dense_8_loss: 0.0358 - val_dense_9_loss: 0.0348
Val Jaccard Similarity: 0.7040279113986752
Epoch 27/200
21560/21560 [==============================] - 13s 596us/step - loss: 0.0749 - dense_8_loss: 0.0379 - dense_9_loss: 0.0369 - val_loss: 0.0694 - val_dense_8_loss: 0.0355 - val_dense_9_loss: 0.0349
Val Jaccard Similarity: 0.7022302134495331
Epoch 28/200
21560/21560 [==============================] - 13s 597us/step - loss: 0.0748 - dense_8_loss: 0.0378 - dense_9_loss: 0.0369 - val_loss: 0.0695 - val_dense_8_loss: 0.0356 - val_dense_9_loss:

21560/21560 [==============================] - 13s 620us/step - loss: 0.0715 - dense_8_loss: 0.0361 - dense_9_loss: 0.0353 - val_loss: 0.0678 - val_dense_8_loss: 0.0346 - val_dense_9_loss: 0.0341
Val Jaccard Similarity: 0.7137328283836303
Epoch 58/200
21560/21560 [==============================] - 13s 596us/step - loss: 0.0712 - dense_8_loss: 0.0361 - dense_9_loss: 0.0351 - val_loss: 0.0682 - val_dense_8_loss: 0.0349 - val_dense_9_loss: 0.0342
Val Jaccard Similarity: 0.7122871008207629
Epoch 59/200
21560/21560 [==============================] - 13s 603us/step - loss: 0.0709 - dense_8_loss: 0.0359 - dense_9_loss: 0.0351 - val_loss: 0.0681 - val_dense_8_loss: 0.0349 - val_dense_9_loss: 0.0342
Val Jaccard Similarity: 0.7147027245898943
Epoch 60/200
21560/21560 [==============================] - 13s 594us/step - loss: 0.0710 - dense_8_loss: 0.0360 - dense_9_loss: 0.0350 - val_loss: 0.0678 - val_dense_8_loss: 0.0349 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7138314684981322
Epoch

Val Jaccard Similarity: 0.7153834683385509
Epoch 90/200
21560/21560 [==============================] - 12s 573us/step - loss: 0.0692 - dense_8_loss: 0.0350 - dense_9_loss: 0.0341 - val_loss: 0.0677 - val_dense_8_loss: 0.0348 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7164511385496671
Epoch 91/200
21560/21560 [==============================] - 13s 609us/step - loss: 0.0691 - dense_8_loss: 0.0350 - dense_9_loss: 0.0341 - val_loss: 0.0682 - val_dense_8_loss: 0.0349 - val_dense_9_loss: 0.0342
Val Jaccard Similarity: 0.7161386179158155
Epoch 92/200
21560/21560 [==============================] - 12s 578us/step - loss: 0.0690 - dense_8_loss: 0.0348 - dense_9_loss: 0.0342 - val_loss: 0.0678 - val_dense_8_loss: 0.0348 - val_dense_9_loss: 0.0340
Val Jaccard Similarity: 0.7158923272246154
Epoch 93/200
21560/21560 [==============================] - 12s 579us/step - loss: 0.0690 - dense_8_loss: 0.0349 - dense_9_loss: 0.0341 - val_loss: 0.0674 - val_dense_8_loss: 0.0346 - val_dense_9_loss:

21560/21560 [==============================] - 12s 578us/step - loss: 0.0677 - dense_8_loss: 0.0344 - dense_9_loss: 0.0334 - val_loss: 0.0679 - val_dense_8_loss: 0.0349 - val_dense_9_loss: 0.0340
Val Jaccard Similarity: 0.7186887668361355
Epoch 123/200
21560/21560 [==============================] - 12s 575us/step - loss: 0.0681 - dense_8_loss: 0.0345 - dense_9_loss: 0.0337 - val_loss: 0.0680 - val_dense_8_loss: 0.0349 - val_dense_9_loss: 0.0340
Val Jaccard Similarity: 0.7192805846165078
Epoch 124/200
21560/21560 [==============================] - 12s 579us/step - loss: 0.0677 - dense_8_loss: 0.0344 - dense_9_loss: 0.0333 - val_loss: 0.0674 - val_dense_8_loss: 0.0346 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7201806663750604
Epoch 125/200
21560/21560 [==============================] - 13s 619us/step - loss: 0.0680 - dense_8_loss: 0.0346 - dense_9_loss: 0.0334 - val_loss: 0.0677 - val_dense_8_loss: 0.0347 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7147511168563762
Ep

KeyboardInterrupt: 

Epoch 98/100
21561/21561 [==============================] - 8s 357us/step - loss: 26.4123 - val_loss: 32.8301
Val Jaccard Similarity: 0.5266293328859964

Epoch 99/100
21561/21561 [==============================] - 8s 356us/step - loss: 26.0379 - val_loss: 32.5830
Val Jaccard Similarity: 0.5592650588400309

Epoch 100/100
21561/21561 [==============================] - 8s 352us/step - loss: 25.8462 - val_loss: 32.7917
Val Jaccard Similarity: 0.5504962474510444

In [ ]:
model_2.fit(
    [train_manip_X, train_numerics_repeated], 
    [train_manip_Y_categorical[0], train_manip_Y_categorical[1]], 
    validation_data=(
        [test_manip_X, test_numerics_repeated], 
        [test_manip_Y_categorical[0], test_manip_Y_categorical[1]]), 
    batch_size=10000, 
    epochs=1000,
    callbacks=[metrics_categorical]
)

Train on 21560 samples, validate on 3804 samples
Epoch 1/1000
21560/21560 [==============================] - 9s 433us/step - loss: 0.0658 - dense_8_loss: 0.0339 - dense_9_loss: 0.0325 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7218930148985786
Epoch 2/1000
21560/21560 [==============================] - 7s 344us/step - loss: 0.0658 - dense_8_loss: 0.0335 - dense_9_loss: 0.0324 - val_loss: 0.0677 - val_dense_8_loss: 0.0340 - val_dense_9_loss: 0.0337
Val Jaccard Similarity: 0.7216997479714511
Epoch 3/1000
21560/21560 [==============================] - 8s 379us/step - loss: 0.0662 - dense_8_loss: 0.0334 - dense_9_loss: 0.0326 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7222091212281733
Epoch 4/1000
21560/21560 [==============================] - 8s 351us/step - loss: 0.0657 - dense_8_loss: 0.0336 - dense_9_loss: 0.0327 - val_loss: 0.0678 - val_dense_8_loss: 0.0341 - val_dense_9_los

21560/21560 [==============================] - 7s 323us/step - loss: 0.0660 - dense_8_loss: 0.0335 - dense_9_loss: 0.0324 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7204436273367022
Epoch 34/1000
21560/21560 [==============================] - 7s 322us/step - loss: 0.0659 - dense_8_loss: 0.0334 - dense_9_loss: 0.0324 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7204316707178481
Epoch 35/1000
21560/21560 [==============================] - 7s 321us/step - loss: 0.0656 - dense_8_loss: 0.0328 - dense_9_loss: 0.0323 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7196071438683778
Epoch 36/1000
21560/21560 [==============================] - 7s 319us/step - loss: 0.0655 - dense_8_loss: 0.0334 - dense_9_loss: 0.0319 - val_loss: 0.0680 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7190405458589565
Epoch 

Val Jaccard Similarity: 0.7191655118886265
Epoch 66/1000
21560/21560 [==============================] - 7s 329us/step - loss: 0.0658 - dense_8_loss: 0.0333 - dense_9_loss: 0.0325 - val_loss: 0.0680 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7190854199426114
Epoch 67/1000
21560/21560 [==============================] - 7s 342us/step - loss: 0.0655 - dense_8_loss: 0.0330 - dense_9_loss: 0.0324 - val_loss: 0.0679 - val_dense_8_loss: 0.0340 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7184766584505693
Epoch 68/1000
21560/21560 [==============================] - 7s 318us/step - loss: 0.0658 - dense_8_loss: 0.0332 - dense_9_loss: 0.0324 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7188523561376191
Epoch 69/1000
21560/21560 [==============================] - 7s 316us/step - loss: 0.0657 - dense_8_loss: 0.0334 - dense_9_loss: 0.0324 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss:

21560/21560 [==============================] - 7s 317us/step - loss: 0.0657 - dense_8_loss: 0.0332 - dense_9_loss: 0.0321 - val_loss: 0.0678 - val_dense_8_loss: 0.0340 - val_dense_9_loss: 0.0337
Val Jaccard Similarity: 0.7184431227844158
Epoch 99/1000
21560/21560 [==============================] - 7s 319us/step - loss: 0.0658 - dense_8_loss: 0.0331 - dense_9_loss: 0.0325 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7180052261602964
Epoch 100/1000
21560/21560 [==============================] - 7s 319us/step - loss: 0.0653 - dense_8_loss: 0.0332 - dense_9_loss: 0.0322 - val_loss: 0.0678 - val_dense_8_loss: 0.0340 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7190393207368677
Epoch 101/1000
21560/21560 [==============================] - 7s 328us/step - loss: 0.0657 - dense_8_loss: 0.0331 - dense_9_loss: 0.0327 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7198568581079412
Epoc

21560/21560 [==============================] - 7s 331us/step - loss: 0.0657 - dense_8_loss: 0.0333 - dense_9_loss: 0.0323 - val_loss: 0.0680 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7210539788952826
Epoch 131/1000
21560/21560 [==============================] - 7s 337us/step - loss: 0.0656 - dense_8_loss: 0.0331 - dense_9_loss: 0.0321 - val_loss: 0.0680 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7206163759041526
Epoch 132/1000
21560/21560 [==============================] - 7s 324us/step - loss: 0.0656 - dense_8_loss: 0.0331 - dense_9_loss: 0.0321 - val_loss: 0.0677 - val_dense_8_loss: 0.0340 - val_dense_9_loss: 0.0337
Val Jaccard Similarity: 0.7188011727101089
Epoch 133/1000
21560/21560 [==============================] - 7s 318us/step - loss: 0.0654 - dense_8_loss: 0.0334 - dense_9_loss: 0.0321 - val_loss: 0.0680 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7181742200519107
Epo

...

21560/21560 [==============================] - 7s 332us/step - loss: 0.0653 - dense_8_loss: 0.0331 - dense_9_loss: 0.0319 - val_loss: 0.0680 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7199684421355531
Epoch 419/1000
21560/21560 [==============================] - 7s 336us/step - loss: 0.0655 - dense_8_loss: 0.0332 - dense_9_loss: 0.0323 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7198070658270792
Epoch 420/1000
21560/21560 [==============================] - 7s 324us/step - loss: 0.0655 - dense_8_loss: 0.0330 - dense_9_loss: 0.0317 - val_loss: 0.0678 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7187744070221613
Epoch 421/1000
21560/21560 [==============================] - 7s 318us/step - loss: 0.0654 - dense_8_loss: 0.0331 - dense_9_loss: 0.0318 - val_loss: 0.0680 - val_dense_8_loss: 0.0342 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7192646513308929
Epo

21560/21560 [==============================] - 7s 321us/step - loss: 0.0655 - dense_8_loss: 0.0330 - dense_9_loss: 0.0320 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0337
Val Jaccard Similarity: 0.7181731606998976
Epoch 451/1000
21560/21560 [==============================] - 7s 322us/step - loss: 0.0653 - dense_8_loss: 0.0331 - dense_9_loss: 0.0324 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7191058400810875
Epoch 452/1000
21560/21560 [==============================] - 7s 325us/step - loss: 0.0652 - dense_8_loss: 0.0335 - dense_9_loss: 0.0327 - val_loss: 0.0680 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.720172348890109
Epoch 453/1000
21560/21560 [==============================] - 7s 318us/step - loss: 0.0656 - dense_8_loss: 0.0333 - dense_9_loss: 0.0325 - val_loss: 0.0680 - val_dense_8_loss: 0.0342 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7192230312776207
Epoc

21560/21560 [==============================] - 7s 337us/step - loss: 0.0652 - dense_8_loss: 0.0332 - dense_9_loss: 0.0319 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7190898485827038
Epoch 483/1000
21560/21560 [==============================] - 7s 326us/step - loss: 0.0657 - dense_8_loss: 0.0338 - dense_9_loss: 0.0326 - val_loss: 0.0680 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7201767810180641
Epoch 484/1000
21560/21560 [==============================] - 7s 318us/step - loss: 0.0652 - dense_8_loss: 0.0329 - dense_9_loss: 0.0322 - val_loss: 0.0677 - val_dense_8_loss: 0.0340 - val_dense_9_loss: 0.0337
Val Jaccard Similarity: 0.7203188502367347
Epoch 485/1000
21560/21560 [==============================] - 7s 322us/step - loss: 0.0654 - dense_8_loss: 0.0332 - dense_9_loss: 0.0323 - val_loss: 0.0681 - val_dense_8_loss: 0.0342 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7196724962300731
Epo

21560/21560 [==============================] - 7s 323us/step - loss: 0.0655 - dense_8_loss: 0.0327 - dense_9_loss: 0.0322 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7160967325386823
Epoch 515/1000
21560/21560 [==============================] - 7s 322us/step - loss: 0.0652 - dense_8_loss: 0.0330 - dense_9_loss: 0.0320 - val_loss: 0.0681 - val_dense_8_loss: 0.0343 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7170263075541713
Epoch 516/1000
21560/21560 [==============================] - 7s 329us/step - loss: 0.0652 - dense_8_loss: 0.0332 - dense_9_loss: 0.0317 - val_loss: 0.0681 - val_dense_8_loss: 0.0342 - val_dense_9_loss: 0.0339
Val Jaccard Similarity: 0.7184489244330416
Epoch 517/1000
21560/21560 [==============================] - 7s 342us/step - loss: 0.0653 - dense_8_loss: 0.0329 - dense_9_loss: 0.0321 - val_loss: 0.0680 - val_dense_8_loss: 0.0342 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7185919136730341
Epo

21560/21560 [==============================] - 7s 335us/step - loss: 0.0654 - dense_8_loss: 0.0328 - dense_9_loss: 0.0320 - val_loss: 0.0678 - val_dense_8_loss: 0.0340 - val_dense_9_loss: 0.0337
Val Jaccard Similarity: 0.7201500162667661
Epoch 547/1000
21560/21560 [==============================] - 7s 329us/step - loss: 0.0654 - dense_8_loss: 0.0336 - dense_9_loss: 0.0326 - val_loss: 0.0679 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0337
Val Jaccard Similarity: 0.7201724589280843
Epoch 548/1000
21560/21560 [==============================] - 7s 321us/step - loss: 0.0654 - dense_8_loss: 0.0334 - dense_9_loss: 0.0318 - val_loss: 0.0678 - val_dense_8_loss: 0.0341 - val_dense_9_loss: 0.0337
Val Jaccard Similarity: 0.7188303349018377
Epoch 549/1000
21560/21560 [==============================] - 7s 321us/step - loss: 0.0654 - dense_8_loss: 0.0334 - dense_9_loss: 0.0323 - val_loss: 0.0678 - val_dense_8_loss: 0.0340 - val_dense_9_loss: 0.0338
Val Jaccard Similarity: 0.7191477915056437
Epo

### Predictions on local test 

In [ ]:
# preds = model_2.predict([test_manip_X, test_manip_X_sentiment_one_hot_repeated])
preds = model_2.predict(
    [
        test_manip_X, test_numerics_repeated
    ]
)
temp_out = pd.concat([
    handlers.get_preds_out(handlers.get_indexes_from_argmax(preds, test_df), test_df), 
    test_df['selected_text']],
    axis=1
)
temp_out.columns = ['predicted_text', 'selected_text']
# temp_out['predicted_text']
temp_out.apply(lambda x: handlers.jaccard(x['predicted_text'], x['selected_text']), axis=1).mean()

- Basic Config with Dropout No Stage: 0.7176681846688269
- Add 200 epochs at 1000 batch: 0.7215144500264917

In [ ]:
# Two Stage Result
neuts = test_df[test_df['sentiment']=='neutral'].reset_index(drop=True)[['text', 'selected_text']]
neuts.columns = ['predicted_text', 'selected_text']

pd.concat(
    [
        temp_out[test_df['sentiment']!='neutral'].reset_index(drop=True),
        neuts
    ]
).reset_index(drop=True).apply(lambda x: handlers.jaccard(x['predicted_text'], x['selected_text']), axis=1).mean()

- Basic Confif with Dropout Static First Stage: 0.7174145968259453
- Add 200 epochs at 1000 batch: 0.7213928829363386






- Check if embedding matrix is correctly created

In [ ]:
# print('Overall Jaccard from Model:')
# print(test_df_1[['text', 'selected_text', 'out_pred_span']].apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean())

# print('Overall Jaccard baseline from predicting the complete text:')
# print(test_df_1[['text', 'selected_text', 'out_pred_span']].apply(lambda x: handlers.jaccard(x['selected_text'], x['text']), axis=1).mean())

# print('Overall Jaccard using model for NOT neutral and baseline for neutral:')
# print(pd.concat([
#     test_df_1[test_df_1['sentiment'] == 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['text']), axis=1),
#     test_df_1[test_df_1['sentiment'] != 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1)
# ]).mean())


# print('Model Jaccard for neutral')
# print(test_df_1[test_df_1['sentiment'] == 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean())

# print('Baseline Jaccard for neutral from predicting complete text')
# print(test_df_1[test_df_1['sentiment'] == 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['text']), axis=1).mean())

# print('Model Jaccard for NOT neutral')
# print(test_df_1[test_df_1['sentiment'] != 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean())

# print('Baseline Jaccard for NOT neutral from predicting complete text')
# print(test_df_1[test_df_1['sentiment'] != 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['text']), axis=1).mean())



### Predictions on Test-Test Data

In [16]:
# (valid_manip_X, valid_numerics_repeated, valid_manip_Y, valid_manip_Y_categorical, valid_df) = data_mainpulation_pipeline.transform(df_objs.df_objects['test'])

In [17]:
# preds = model_2.predict(
#     [
#         valid_manip_X, valid_numerics_repeated
#     ]
# )
# df_objs.df_objects['test']['out_pred_span'] = handlers.get_preds_out(handlers.get_indexes_from_argmax(preds, df_objs.df_objects['test']), df_objs.df_objects['test'])
# final_out = df_objs.df_objects['test'][['textID','out_pred_span']]
# final_out.columns = ['textID', 'selected_text']

In [41]:
# final_out.head(5)

In [42]:
# final_out.to_csv('submission.csv', index=False)